In [ ]:
import math
import random
import os
import cv2
import time
import warnings
import numpy as np
from skimage.io import imread, imsave, imshow
from multiprocessing import Pool

warnings.simplefilter('ignore')

In [ ]:
def get_filenames(datapath='lane_marking_examples'):
    filenames = []
    for top, dirs, files in os.walk(datapath):
        filenames.extend([os.path.join(top, _file) for _file in files])
    filenames.sort()

    x_paths = [x for x in filenames if not x.endswith('bin.png')]
    y_paths = [x for x in filenames if x.endswith('bin.png')]

    return x_paths, y_paths

x_paths, y_paths = get_filenames()

In [ ]:
def split_image(xfilepath, yfilepath, ind):
    IMG_ORIG_H = 2710
    IMG_ORIG_W = 3384
    IMG_SPLT_H = 192
    IMG_SPLT_W = 192
    STEP = 96
    
    save_path_img = 'data_prepaired/road0203/ColorImage/Record0/'
    save_path_lbl = 'data_prepaired/road0203/Label/Record0/'
    
    img = cv2.imread(xfilepath)
    lbl = cv2.imread(yfilepath)
    no_white = lbl.copy()
    no_white[no_white == 255] = 0
    eq_lbl = no_white != np.array([0, 0, 0])
    
    fname_img = xfilepath.split('/')[-1][:-4] 
    fname_lbl = yfilepath.split('/')[-1][:-4]
    
    w, h = 0, 0
    _f = True
    total = 0
    while h + IMG_SPLT_H < IMG_ORIG_H and _f:
        w = 0
        while w + IMG_SPLT_W < IMG_ORIG_W and _f:
            # unqs = np.unique(lbl[h:h+IMG_SPLT_H, w:w+IMG_SPLT_W])
            if eq_lbl[h:h+IMG_SPLT_H, w:w+IMG_SPLT_W].sum() / 36864 > 0.4: # or unqs.__len__() > 5:
                crop_img = img[h:h+IMG_SPLT_H, w:w+IMG_SPLT_W]
                crop_lbl = lbl[h:h+IMG_SPLT_H, w:w+IMG_SPLT_W]
                imsave(save_path_img + fname_img + f'_{str(h).zfill(4)}_{str(w).zfill(4)}.png', crop_img)
                imsave(save_path_lbl + fname_lbl + f'_{str(h).zfill(4)}_{str(w).zfill(4)} bin.png', crop_lbl)
                # _f = False
                total += 1
            w += STEP
        h += STEP
    # print(f'Total images saved: {total}')
    print(f'Image by index {str(ind).zfill(4)} splitted', end='\r')

In [ ]:
# pics_to_split = [5750, 8990, 3328, 2210, 6160, 4150, 4200, 10040, 10080, 10135]
pics_to_split = [x for x in range(0, 10464, 10)]

for ind in pics_to_split:
    split_image(x_paths[ind], y_paths[ind], ind)

In [ ]:
p = random.choice(y_paths)
print(y_paths.index(p))
imshow(cv2.imread(p))

In [ ]:
imshow(cv2.imread(y_paths[7360]))